In [55]:
import numpy as np
import pandas as pd

In [56]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer

In [70]:
data=pd.read_csv('covid_toy.csv')
data.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [58]:
df.isna().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

##### Usual way

In [59]:
# adding misising fever value using simple imputer
si=SimpleImputer()
df['fever']=si.fit_transform(df[['fever']])

In [60]:
df.isna().sum()

age          0
gender       0
fever        0
cough        0
city         0
has_covid    0
dtype: int64

In [61]:
ohe=OneHotEncoder(sparse_output=False)
encoded_columns=ohe.fit_transform(df[['gender','city']])
column_names=ohe.get_feature_names_out(['gender', 'city'])
encoded_df=pd.DataFrame(encoded_columns,columns=column_names)
df = pd.concat([df.drop(columns=['gender', 'city']), encoded_df], axis=1)


In [63]:
df.sample(5)

,age,fever,cough,has_covid,gender_Female,gender_Male,city_Bangalore,city_Delhi,city_Kolkata,city_Mumbai
96,51,101.0,Strong,Yes,1.0,0.0,0.0,0.0,1.0,0.0
86,25,104.0,Mild,Yes,0.0,1.0,1.0,0.0,0.0,0.0
98,5,98.0,Strong,No,1.0,0.0,0.0,0.0,0.0,1.0
13,64,102.0,Mild,Yes,0.0,1.0,1.0,0.0,0.0,0.0
73,34,98.0,Strong,Yes,0.0,1.0,0.0,0.0,1.0,0.0


In [66]:
# ordinal encoding
oe=OrdinalEncoder(categories=[['Mild','Strong']])
df['cough']=oe.fit_transform(df[['cough']])

In [67]:
df.head()

,age,fever,cough,has_covid,gender_Female,gender_Male,city_Bangalore,city_Delhi,city_Kolkata,city_Mumbai
0,60,103.0,0.0,No,0.0,1.0,0.0,0.0,1.0,0.0
1,27,100.0,0.0,Yes,0.0,1.0,0.0,1.0,0.0,0.0
2,42,101.0,0.0,No,0.0,1.0,0.0,1.0,0.0,0.0
3,31,98.0,0.0,No,1.0,0.0,0.0,0.0,1.0,0.0
4,65,101.0,0.0,No,1.0,0.0,0.0,0.0,0.0,1.0


##### Using column transformer

In [68]:
from sklearn.compose import ColumnTransformer


In [69]:
transformer=ColumnTransformer(transformers=[
    ('tsi',SimpleImputer(),['fever']),
    ('toe',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tohe',OneHotEncoder(sparse_output=False),['gender','city'])
],remainder='passthrough')

In [72]:
transformer.fit_transform(data).shape

(100, 10)